
#### vehicle state
$$
\mathbf{x_k} =
        \left[\begin{array}{c}
        \mathbf{p_k} \\
        \mathbf{v_k} \\
        \mathbf{q_k} \\
        \end{array}\right] \quad
        \in
        R^{10}
$$
$\mathbf{p_k}$ is 3D Position.  $\mathbf{v_k}$ is 3D velocity. $\mathbf{q_k}$ is 4D quaternion 
#### motion model input
$$
\mathbf{u_k} =
        \left[\begin{array}{c}
        \mathbf{f_k} \\
        \mathbf{w_k} \\
        \end{array}\right] \quad
        \in
        R^{6}
$$

### Measurement Model
$$ 
\begin{align}
\mathbf{y_k} &= \mathbf{h}(\mathbf{x_k}) + \mathbf{\upsilon_{k}} \\
&= \mathbf{H_k} \mathbf{x_k} + \mathbf{\upsilon_{k}} \\
&= \left[\begin{array}{c} \mathbf{1} & \mathbf{0} & \mathbf{0}\end{array}\right] \mathbf{x_k} + \mathbf {\upsilon_{k}}
\end{align} \\
\mathbf{\upsilon_{k}} \sim N(\mathbf{0}, R_{GNSS}) \\
\mathbf{\upsilon_{k}} \sim N(\mathbf{0}, R_{LIDAR}) \\
$$

\begin{equation}
r′=C_{ns} r
\end{equation}

\begin{equation}
C_{ns} = \begin{bmatrix} q_0^2+q_1^2-q_2^2-q_3^2 & 2(q_1q_2-q_0q_3) & 2(q_1q_3+q_0q_2) \\ 2(q_1q_2+q_0q_3) & q_0^2-q_1^2+q_2^2-q_3^2 & 2(q_2q_3-q_0q_1) \\ 2(q_1q_3-q_0q_2) & 2(q_2q_3+q_0q_1) & q_0^2-q_1^2-q_2^2+q_3^2 \end{bmatrix}
\end{equation}


## Motion Model

Position $$ \mathbf{p_k} = \mathbf{p_{k-1}} + \Delta t \mathbf{v_{k-1}} + \frac{\Delta t ^ 2}{2}( \mathbf{C_{ns}}\mathbf{f_{k-1}}+\mathbf{g} )$$
Velocity $$ \mathbf{v_k} = \mathbf{v_{k-1}} + \Delta t ( \mathbf{C_{ns}}\mathbf{f_{k-1}}+\mathbf{g} )$$
Orientation $$ \mathbf{q_{k}} = \mathbf{q_{k-1}} \otimes \mathbf{q} ( \mathbf{w_{k-1}} \Delta t)
= \Omega ( \mathbf{q}(\mathbf{w_{k-1}} \Delta t )) \mathbf{q_{k_1}} $$
where ...
$$
\mathbf{C_{ns}} = \mathbf{C_{ns}}(\mathbf{q_{k-1}}) \\
\Omega \Biggl( \left[\begin{array}{c}
        q_w \\
        \mathbf{q_v} \\
        \end{array}\right] \Biggr) = q_{w}\mathbf{1}
        + \left[\begin{array}{c}
        0 & -\mathbf{q^T_v}\\
        \mathbf{q_v} & -[\mathbf{q_v}]_x\\
        \end{array}\right] \quad
\mathbf{q}(\theta) = \left[\begin{array}{c}
        \cos \frac{|\theta|}{2} \\
        \frac{\theta}{|\theta|} \sin \frac{|\theta|}{2} \\
        \end{array}\right]
$$

## Linearized Motion Model
#### Error State
$$
\delta \mathbf{x_k} = \left[\begin{array}{c}
        \delta \mathbf{p_k} \\
        \delta \mathbf{v_k} \\
        \delta \Phi_{k} \\
        \end{array}\right] \in R^9
$$
$\delta \Phi_{k}$ is 3x1 rotation error  
#### Error Dynamics
$$
\delta \mathbf{x_k} = \mathbf{F_{k-1}} \delta \mathbf{x_{k-1}} + \mathbf{L_{k-1}} \mathbf{n_{k-1}}
$$
where ...
$$
\mathbf{F_{k-1}} = \left[\begin{array}{c}
        \mathbf{1} & \mathbf{1}\Delta t & \mathbf{0} \\
        \mathbf{0} & \mathbf{1} & -[\mathbf{C_{ns}} \mathbf{f_{k-1}}]_X \Delta t\\
        \mathbf{0} & \mathbf{0} & \mathbf{1} \\
        \end{array}\right]
\\
\mathbf{L_{k-1}} = \left[\begin{array}{c}
        \mathbf{0} & \mathbf{0} \\
        \mathbf{1} & \mathbf{0} \\
        \mathbf{0} & \mathbf{1} \\
        \end{array}\right]
\\
\mathbf{n_k} \sim N (\mathbf{0}, \mathbf{Q_k})
\sim N \Biggl( \mathbf{0} , \Delta t^2 \left[\begin{array}{c}
        \sigma_{acc}^2 & \mathbf{0} \\
        \mathbf{0} & \sigma_{gyro}^2\\
        \end{array}\right]
        \Biggr)
$$


## Error State EKF
### Prediction
#### 1. Update state with IMU inputs
$$
\mathbf{\breve{x_k}} =
        \left[\begin{array}{c}
        \mathbf{\breve{p_k}} \\
        \mathbf{\breve{v_k}} \\
        \mathbf{\breve{q_k}} \\
        \end{array}\right] \quad
$$
$$
\begin{align}
\breve{\mathbf{p_k}} &= \mathbf{p_{k-1}} + \Delta t \mathbf{v_{k-1}} + \frac{\Delta t ^ 2}{2}( \mathbf{C_{ns}}\mathbf{f_{k-1}}+\mathbf{g_n} ) \\
\mathbf{\breve{v_k}} &= \mathbf{v_{k-1}} + \Delta t ( {C_{ns}}\mathbf{f_{k-1}}+\mathbf{g_n} ) \\
\mathbf{\breve{q_{k}}} &= \mathbf{q_{k-1}} \otimes \mathbf{q} ( \mathbf{w_{k-1}} \Delta t) 
= \Omega ( \mathbf{q}(\mathbf{w_{k-1}} \Delta t )) \mathbf{q_{k_1}} \\
\end{align}
$$
Multiply quaternion from right.
#### 2. Propagate uncertainty
$$
\mathbf{\breve{P_k}} = \mathbf{F_{k-1}} \mathbf{P_{k-1}} \mathbf{F^T_{k-1}} + \mathbf{L_{k-1}} \mathbf{Q_{k-1}} \mathbf{L^T_{k-1}}
$$
### Correction
#### 3 If GNSS or LIDAR position available:
##### 3.1. Compute Kalman gain
$$
\mathbf{K_k} = \mathbf{\breve{P_k}} \mathbf{H^T_k} (\mathbf{H_k} \mathbf{\breve{P_k}} \mathbf{H^T_k} + \mathbf{R})^{-1}
$$
$\mathbf{R}$ is $\mathbf{R_{GNSS}}$ or $\mathbf{R_{LIDAR}}$
#### 3.2. Compute Error State
$$
\delta \mathbf{x_k} = \mathbf{K_k} ( \mathbf{y_k} - \mathbf{\breve{p_k}} )
$$
#### 3.3. Correct predicted state
$$
\begin{align}
\mathbf{\hat{p_k}} &= \mathbf{\breve{p_k}} + \delta \mathbf{p_k} \\
\mathbf{\hat{v_k}} &= \mathbf{\breve{v_k}} + \delta \mathbf{v_k} \\
\mathbf{\hat{q_k}} &= \mathbf{q}(\delta \Phi) \otimes \mathbf{\breve{q_k}}
\end{align}
$$
$\delta \Phi$ is global orientation error  
Multiply quaternion from left.
#### 3.4 Compute corrected covariance
$$
\mathbf{\hat{P_k}} = (\mathbf{1} - \mathbf{K_k}\mathbf{H_k})\mathbf{\breve{P_k}}
$$